## Import Packages

In [10]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
import tensorflow as tf
import ntpath
import papermill

alpha = 2
print("alpha:")
print(alpha)


## Import CSVs

In [11]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed\\regression_5d'
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = ntpath.basename(file)
    dfs.append(df)

df = dfs[4]
print("Stock: ")
print(df.name)
print("alpha:")
print(alpha)
srfs

C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\mlpLearning\..\..\Datasets\processed\regression_5d
Stock: 
AWK.csv


# Splitting dataset in dependent and independent variables

In [12]:
x = np.array(df.iloc[:, 1:-1].values, dtype=np.float)
y = np.array(df.iloc[:, -1:].values)


# Splitting dataset (training + testing)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

print("Train - Independent Variables")
print(x_train)
print("Test - Independent Variables")
print(x_test)
print("Train - Dependent Variables")
print(y_train)
print("Test - Dependent Variables")
print(y_test)


Train - Independent Variables
[[  1.99   1.87   1.66 ...   0.28   0.51  91.79]
 [  1.87   1.66   2.65 ...   0.52   0.78  92.56]
 [  1.66   2.65   3.02 ...   0.55   0.81  92.61]
 ...
 [ 58.73  58.48  58.2  ...   0.58   1.12 149.11]
 [ 58.48  58.2   61.02 ...  -0.75   2.58 142.42]
 [ 58.2   61.02  60.34 ...  -0.96   2.62 145.87]]
Test - Independent Variables
[[ 6.10e+01  6.03e+01  5.36e+01  5.71e+01  5.77e+01 -9.80e-01  2.62e+00
   1.47e+02]
 [ 6.03e+01  5.36e+01  5.71e+01  5.77e+01  5.65e+01 -3.51e-01  2.15e+00
   1.45e+02]
 [ 5.36e+01  5.71e+01  5.77e+01  5.65e+01  5.78e+01  7.72e-01  1.53e+00
   1.47e+02]
 [ 5.71e+01  5.77e+01  5.65e+01  5.78e+01  5.72e+01  3.10e-02  4.59e-01
   1.46e+02]
 [ 5.77e+01  5.65e+01  5.78e+01  5.72e+01  5.51e+01 -4.49e-01  9.80e-01
   1.44e+02]
 [ 5.65e+01  5.78e+01  5.72e+01  5.51e+01  5.55e+01 -4.70e-01  9.99e-01
   1.44e+02]
 [ 5.78e+01  5.72e+01  5.51e+01  5.55e+01  5.49e+01 -7.56e-01  1.17e+00
   1.44e+02]
 [ 5.72e+01  5.51e+01  5.55e+01  5.49e+01  5.5

# Building ANN with MLP

In [14]:
ann = tf.keras.models.Sequential()
# hidden layer
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

optimizer = tf.keras.optimizers.Adam(lr=0.02)
ann.compile(optimizer = optimizer, loss = 'mean_squared_error')

ann.fit(x_train, y_train, batch_size = 32, epochs = 150)

weights = ann.get_weights()

Train on 397 samples
Epoch 1/150
397/397 [==============================] - 1s 2ms/sample - loss: 2507.4771
Epoch 2/150
397/397 [==============================] - 0s 80us/sample - loss: 577.1441
Epoch 3/150
397/397 [==============================] - 0s 90us/sample - loss: 330.3054
Epoch 4/150
397/397 [==============================] - 0s 90us/sample - loss: 219.8986
Epoch 5/150
397/397 [==============================] - 0s 95us/sample - loss: 165.5060
Epoch 6/150
397/397 [==============================] - 0s 105us/sample - loss: 125.9952
Epoch 7/150
397/397 [==============================] - 0s 100us/sample - loss: 97.9869

# Predicting Test Set Results

In [15]:
y_pred = ann.predict(x_test)
# boolean result of whether predicted probability is larger than 0.5
# result > 0.5 = 1, result <= 0.5 = 0
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


[[148.35 145.31]
 [145.71 146.56]
 [144.74 146.  ]
 [145.79 143.92]
 [143.38 144.28]
 [143.22 143.64]
 [143.87 143.88]
 [143.64 143.53]
 [143.11 140.9 ]
 [140.6  141.37]
 [140.47 140.79]
 [141.17 141.34]
 [141.25 140.17]
 [139.87 144.15]
 [143.23 142.48]
 [142.88 143.17]
 [141.64 140.77]
 [140.   143.2 ]
 [142.4  140.77]
 [141.25 141.1 ]
 [140.18 142.88]
 [142.35 143.59]
 [143.83 142.77]
 [141.89 141.02]
 [139.89 138.28]
 [137.76 139.72]
 [139.41 140.17]
 [141.04 138.05]
 [138.11 140.66]
 [139.25 144.23]
 [144.38 144.55]
 [144.47 143.7 ]
 [141.78 144.88]
 [143.2  148.57]
 [148.11 148.96]
 [148.86 151.89]
 [149.98 152.49]
 [151.31 154.51]
 [152.99 155.85]
 [154.55 155.75]
 [154.6  158.32]
 [156.59 156.08]
 [155.57 155.76]
 [154.05 155.86]
 [155.   155.72]
 [155.63 154.52]
 [153.88 153.15]
 [152.34 152.25]
 [151.74 153.61]
 [153.33 154.08]
 [154.12 154.01]
 [153.25 154.62]
 [153.4  153.  ]
 [152.38 152.16]
 [151.12 150.51]
 [150.08 154.3 ]
 [153.74 157.1 ]
 [157.46 155.41]
 [154.71 158.2

# Obtaining Confusion Matrix and Accuracy Score for predictions

In [16]:
from sklearn.metrics import mean_absolute_error


print()
print("Mean Absolute Error: ")
mean_absolute_error(y_test, y_pred)


Mean Absolute Error: 


1.6927880981445316

# Method execution